In [1]:
import Pkg; Pkg.activate(joinpath(@__DIR__,"../../../..")); Pkg.instantiate()

  Activating environment at `~/Desktop/PHD/MultiAgentOptimization/julia_bots_ws/Project.toml`
Precompiling project...
  ✓ Parsers
  ✓ JSON
  ✓ Conda
  ✓ PyCall
  ✓ RobotOS
  5 dependencies successfully precompiled in 12 seconds (9 already precompiled)
  3 dependencies precompiled but different versions are currently loaded. Restart julia to access the new versions


In [2]:
#!/usr/bin/env julia167
#!/usr/bin/python
# Change to 'julia' instead of 'julia167' if you don't have a symlink setup

using RobotOS
using iLQGameSolver
using SparseArrays
using LinearAlgebra
using StaticArrays


@rosimport geometry_msgs.msg: Twist, Pose
@rosimport nav_msgs.msg.Odometry
rostypegen()
using .geometry_msgs.msg: Twist, Pose
using .nav_msgs.msg: Odometry

┌ Info: Precompiling RobotOS [22415677-39a4-5241-a37a-00beabbbdae8]
└ @ Base loading.jl:1342


In [3]:
# Setup the problem

dt = 0.1                    # Step size [s]
tf = 35.0                    # Horizon [s]
N = Int(tf/dt)         # Number of steps (knot points)
println(N)

# Define cost matrices 
nx = 3 
nu = 2
Nplayer = 2

Nu = nu * Nplayer
Nx = nx * Nplayer

Q1 = sparse(zeros(Nx,Nx))     # State cost for agent 1
Q1[1:nx,1:nx] .= 2.0*I(nx)
Qn1 = 5.0*Q1                    # Terminal cost for agent 1

Q2 = sparse(zeros(Nx,Nx))     # State cost for agent 2
Q2[nx+1:2*nx,nx+1:2*nx] .= 2.0*I(nx)
Qn2 = 5.0*Q2                    # Terminal cost for agent 2

# Q3 = sparse(zeros(Nx,Nx))     # State cost for agent 2
# Q3[2*nx+1:3*nx,2*nx+1:3*nx] .= 3.0*I(nx)
# Qn3 = Q3                    # Terminal cost for agent 2

R11 = sparse(5.0*I(2))              # Control cost for player 1
R12 = sparse(0.0*I(2))     # Control cost for player 1 associated with player 2's controls
# R13 = sparse(0.0*I(2))     # Control cost for player 2 associated with player 1's controls
R21 = sparse(0.0*I(2))     # Control cost for player 2 associated with player 1's controls
R22 = sparse(5.0*I(2))              # Contorl cost for player 2
# R23 = sparse(0.0*I(2))     # Control cost for player 2 associated with player 1's controls
# R31 = sparse(0.0*I(2))     # Control cost for player 2 associated with player 1's controls
# R32 = sparse(0.0*I(2))     # Control cost for player 2 associated with player 1's controls
# R33 = sparse(1.0*I(2))     # Control cost for player 2 associated with player 1's controls

dmax = 0.5                  # Distance that both agents should keep between each other [m]
ρ = 500.0                   # Penalty factor for violating the distance constraint

# Q's are stacked vertically
Q = sparse(zeros(Float32, Nx*Nplayer, Nx))
# @show size([Q1; Q2; Q3]), size(Q)
Q .= [Q1; Q2]
#Q .= [Q1; Q2; Q3]

# Qn's are stacked vertically
Qn = sparse(zeros(Float32, Nx*Nplayer, Nx))
Qn .= [Qn1; Qn2]
#Qn .= [Qn1; Qn2; Qn3]

# R's are stacked as a matrix
R = sparse(zeros(Float32, Nu, Nu))
R .= [R11 R12; R21 R22]
#R .= [R11 R12 R13; R21 R22 R23; R31 R32 R33]

NHor = 60
tol = 1e-2

game = iLQGameSolver.GameSetup(nx, nu, Nplayer, Q, R, Qn, dt, tf, NHor, dmax, ρ, tol)

solver = iLQGameSolver.iLQSetup(Nx, Nu, Nplayer, NHor);

350


In [4]:
function nominal_trajectory(x0, xgoal, N,dt)
    xref = [fill(NaN, length(x0)) for k = 1:N]
    xref[1] .= x0
    @show x0
    V1 = (xgoal[1:2]-x0[1:2])/((N-1)*dt)
    V2 = (xgoal[4:5]-x0[4:5])/((N-1)*dt)
    #Xref[1][4:5] = V
    for k = 1:N-1
        #xref[k+1] .= x0
        #xref[k][4:5] .= V
        xref[k+1][3] = x0[3]
        xref[k+1][6] = x0[6]
        xref[k+1][1:2] .= (xref[k][1:2]) + dt*V1
        xref[k+1][4:5] .= (xref[k][4:5]) + dt*V2
    end

    return SVector{6}.(xref)
end

nominal_trajectory (generic function with 1 method)

In [5]:
x0 = [   2.5; 0.0; pi/2; 
        0.0; 2.5; 0.0]

xgoal = [   2.5; 5.0; pi/2; 
        5.0; 2.5; 0.0]   
xref = nominal_trajectory(x0, xgoal, N, dt)

umax = [0.3, 0.3, 
        0.3, 0.3]   

umin = [-2.0, -2.0, 
        -2.0, -2.0]

ugoal = [   0.0, 0.0, 
            0.0, 0.0]  
            
solver.P .= rand(NHor, Nu, Nx)*0.01
solver.α = rand(NHor, Nu)*0.01

game.x0 .= x0
game.xref .= xref
game.umin .= umin
game.umax .= umax
game.uf .= ugoal;

x0 = [2.5, 0.0, 1.5707963267948966, 0.0, 2.5, 0.0]


In [6]:
xref

350-element Vector{SVector{6, Float64}}:
 [2.5, 0.0, 1.5707963267948966, 0.0, 2.5, 0.0]
 [2.5, 0.014326647564469917, 1.5707963267948966, 0.014326647564469917, 2.5, 0.0]
 [2.5, 0.028653295128939833, 1.5707963267948966, 0.028653295128939833, 2.5, 0.0]
 [2.5, 0.04297994269340975, 1.5707963267948966, 0.04297994269340975, 2.5, 0.0]
 [2.5, 0.05730659025787967, 1.5707963267948966, 0.05730659025787967, 2.5, 0.0]
 [2.5, 0.07163323782234958, 1.5707963267948966, 0.07163323782234958, 2.5, 0.0]
 [2.5, 0.08595988538681949, 1.5707963267948966, 0.08595988538681949, 2.5, 0.0]
 [2.5, 0.1002865329512894, 1.5707963267948966, 0.1002865329512894, 2.5, 0.0]
 [2.5, 0.11461318051575932, 1.5707963267948966, 0.11461318051575932, 2.5, 0.0]
 [2.5, 0.12893982808022925, 1.5707963267948966, 0.12893982808022925, 2.5, 0.0]
 [2.5, 0.14326647564469916, 1.5707963267948966, 0.14326647564469916, 2.5, 0.0]
 [2.5, 0.15759312320916907, 1.5707963267948966, 0.15759312320916907, 2.5, 0.0]
 [2.5, 0.17191977077363899, 1.57079632679

In [7]:
mutable struct BotPosition
    x::Float32
end

In [8]:
mutable struct BotsState
    x::Vector{Float32}
end

In [9]:
botsSt0 = BotsState(x0[1:3])
tbot0State = botsSt0.x
botsSt1 = BotsState(x0[4:6])
tbot1State = botsSt1.x

3-element Vector{Float32}:
 0.0
 2.5
 0.0

In [10]:
function tb0_callback(msg)
    #loginfo("$(RobotOS.get_caller_id()) I heard $(msg.data)")
    #println(msg.pose.pose.position)
    xpos = msg.pose.pose.position.x
    ypos = msg.pose.pose.position.y

    qx = msg.pose.pose.orientation.x
    qy = msg.pose.pose.orientation.y
    qz = msg.pose.pose.orientation.z
    qw = msg.pose.pose.orientation.w

    theta = atan(2.0 * (qw * qz + qx * qy), 1.0 - 2.0 * (qy * qy + qz * qz))
    tbot0State .= [xpos; ypos; theta]
end

function tb1_callback(msg)
    xpos = msg.pose.pose.position.x
    ypos = msg.pose.pose.position.y

    qx = msg.pose.pose.orientation.x
    qy = msg.pose.pose.orientation.y
    qz = msg.pose.pose.orientation.z
    qw = msg.pose.pose.orientation.w

    theta = atan(2.0 * (qw * qz + qx * qy), 1.0 - 2.0 * (qy * qy + qz * qz))
    tbot1State .= [xpos; ypos; theta]
end

function commands(pub0, pub1)
    agent1 = Twist()
    agent2 = Twist()

    solver.P = rand(NHor, Nu, Nx)*0.01
    solver.α = rand(NHor, Nu)*0.01
    a = 0

    X = zeros(N, Nx) 
    U = zeros(N-1, Nu)
    X[1,:] = game.x0
    println("Game Setup done!")
    rate = Rate(10) # 10 Hz
    tstart = time_ns()
    for k = 1:N-1-NHor
        #println(tbot0State)
        #println("type: ", typeof([botsSt0.x; botsSt1.x]))
        #println("type: ", X[k])
        X[k,:] .= [tbot0State; tbot1State]
        #println(X[k,6])
        xₜ, uₜ = iLQGameSolver.solveILQGame(game, solver, iLQGameSolver.diffDrive3D, iLQGameSolver.costPointMass, X[k,:], k, false)
        X[k+1,:], U[k,:] = xₜ[2,:,:], uₜ[1,:,:]
        # println("State: ", X[k,:])
        # println("Input: ", U[k,:])
        agent1.linear.x = U[k,1]#*cos(rad2deg(xₜ[i,3]))
        agent1.linear.y = 0.0#uₜ[i,1]*sin(rad2deg(xₜ[i,3]))
        agent1.angular.z = U[k,2]#uₜ[i,2]


        agent2.linear.x = U[k,3]#*cos(rad2deg(xₜ[i,6]))
        agent2.linear.y = 0.0#*sin(rad2deg(xₜ[i,6]))
        agent2.angular.z = U[k,4]#uₜ[i,4]

        publish(pub0, agent1)
        publish(pub1, agent2)
        rossleep(rate)
    end
    println(N)
    for k = N-NHor:N-1
        X[k,:] .= [tbot0State; tbot1State]
        xₜ, uₜ = iLQGameSolver.solveILQGame(game, solver, iLQGameSolver.diffDrive3D, iLQGameSolver.costPointMass, X[k,:], k, true)
        X[k+1,:], U[k,:] = xₜ[2,:,:], uₜ[1,:,:]

        agent1.linear.x = U[k,1]#*cos(rad2deg(xₜ[i,3]))
        agent1.linear.y = 0.0#uₜ[i,1]*sin(rad2deg(xₜ[i,3]))
        agent1.angular.z = U[k,2]#uₜ[i,2]


        agent2.linear.x = U[k,3]#*cos(rad2deg(xₜ[i,6]))
        agent2.linear.y = 0.0#*sin(rad2deg(xₜ[i,6]))
        agent2.angular.z = U[k,4]#uₜ[i,4]

        publish(pub0, agent1)
        publish(pub1, agent2)
        rossleep(rate)
        #println(k)
    end
    
    tend = time_ns()
    rossleep(rate)

    for k = 1:10
        X[k,:] .= [tbot0State; tbot1State]
        #println("Done")
        agent1.linear.x = 0.0#*cos(rad2deg(xₜ[i,3]))
        agent1.linear.y = 0.0#uₜ[i,1]*sin(rad2deg(xₜ[i,3]))
        agent1.angular.z = 0.0#uₜ[i,2]


        agent2.linear.x = 0.0#*cos(rad2deg(xₜ[i,6]))
        agent2.linear.y = 0.0#*sin(rad2deg(xₜ[i,6]))
        agent2.angular.z = 0.0#uₜ[i,4]

        publish(pub0, agent1)
        publish(pub1, agent2)
        rossleep(rate)
    end

    rate = N / (tend - tstart) * 1e9
    println("Controller ran at $rate Hz")
    
    return X;
    
end

function main()
    init_node("turtlebot3_cmd") # node name
    pub0 = Publisher{Twist}("/tb3_0/cmd_vel", queue_size=1) # topic name
    pub1 = Publisher{Twist}("/tb3_1/cmd_vel", queue_size=1) # topic name

    tbb1_odom = Subscriber{Odometry}("/tb3_0/odom", tb0_callback; queue_size=2)
    tbb2_odom = Subscriber{Odometry}("/tb3_1/odom", tb1_callback; queue_size=2)

    X = commands(pub0, pub1)

    #spin()
    #return X
end

if !isinteractive()
    main()
end


In [11]:
main()

Game Setup done!
350
Controller ran at 7.943618929798841 Hz


350×6 Matrix{Float64}:
 2.49925  4.89483    1.56838  4.89534    2.51208  -0.00987797
 2.49926  4.9021     1.56837  4.90258    2.51201  -0.0095473
 2.49929  4.91232    1.56792  4.90903    2.51198  -0.00992538
 2.49931  4.92285    1.5677   4.91389    2.51195  -0.0103308
 2.49932  4.92952    1.56756  4.91714    2.51193  -0.0106421
 2.49932  4.93247    1.56753  4.9188     2.51192  -0.0108228
 2.49933  4.93266    1.56751  4.91909    2.51192  -0.0108679
 2.49933  4.93272    1.5675   4.9191     2.51192  -0.0108782
 2.49933  4.93277    1.56748  4.91913    2.51192  -0.0108916
 2.49933  4.93283    1.56747  4.91915    2.51191  -0.0109054
 2.49781  0.0689348  1.59458  0.0689153  2.50226   0.0249934
 2.49738  0.0875331  1.59325  0.0875125  2.50271   0.023576
 2.49698  0.10567    1.5915   0.105648   2.50313   0.0217407
 ⋮                                                 ⋮
 2.49894  4.78504    1.56807  4.78549    2.51341  -0.0127588
 2.49896  4.79606    1.56813  4.79651    2.51327  -0.0123105
 2.49899

In [ ]:
X

In [ ]:
testX = BotsState(x0[1:3])
tbotXtest = testX.x

In [ ]:
tbotXtest

In [ ]:
function changeX()
    tbotXtest .= [1; 2; 3]
end

function showX()
    println(tbotXtest)
end

In [ ]:
changeX()
showX()

In [ ]:
tbotXtest